<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/Face%20Mask%20Detection/Face_Mask_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle

In [2]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

 97% 320M/330M [00:02<00:00, 138MB/s]
100% 330M/330M [00:02<00:00, 147MB/s]


In [3]:
!unzip --qq face-mask-12k-images-dataset.zip

In [4]:
import tensorflow as tf
from tensorflow.keras import layers,optimizers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import os
import datetime

In [5]:
batch_size = 12
epoch = 10
lr = 0.001
width = height = 178

In [7]:
data_generator = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip = True,
)

train_data = data_generator.flow_from_directory(
    '/content/Face Mask Dataset/Train',
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
)

val_data = data_generator.flow_from_directory(
    '/content/Face Mask Dataset/Validation',
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = True,
)

test_data = data_generator.flow_from_directory(
    '/content/Face Mask Dataset/Test',
    target_size = (width,height),
    class_mode = 'categorical',
    batch_size = batch_size,
    shuffle = False,
)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 992 images belonging to 2 classes.


In [8]:
ResNet = tf.keras.applications.ResNet50V2(include_top=False, pooling='avg', weights='imagenet',
input_shape=(width, height, 3))

94683136/94668760 [==============================] - 1s 0us/step


In [9]:
model = tf.keras.Sequential([
        ResNet,
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(2, activation='sigmoid'),              
])

In [10]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.binary_crossentropy,
              metrics=['accuracy'],
              )

In [11]:
model.fit(train_data,
          steps_per_epoch = train_data.samples/batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples/batch_size,
          epochs = 1,
          )

833/833 [==============================] - 237s 237ms/step - loss: 0.1293 - accuracy: 0.9557 - val_loss: 0.0322 - val_accuracy: 0.9925


In [12]:
model.evaluate(test_data)

83/83 [==============================] - 6s 77ms/step - loss: 0.0544 - accuracy: 0.9829


[0.05435765162110329, 0.9828628897666931]

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
model.save('FaceMask.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [15]:
from sklearn.metrics import classification_report, confusion_matrix

Y_pred = model.predict(test_data)
y_pred = np.argmax(Y_pred, axis = 1)
print('confusion Matrix')
print(confusion_matrix(test_data.classes, y_pred))

target_names = list(test_data.class_indices.keys())
print('Classification Report')
print(classification_report(test_data.classes, y_pred, target_names=target_names))

confusion Matrix
[[474   9]
 [ 10 499]]
Classification Report
              precision    recall  f1-score   support

    WithMask       0.98      0.98      0.98       483
 WithoutMask       0.98      0.98      0.98       509

    accuracy                           0.98       992
   macro avg       0.98      0.98      0.98       992
weighted avg       0.98      0.98      0.98       992

